# **Content-based recommendation system for Netflix**

## Загрузка инструментов

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

## Загрузка данных

Реализуем несложную рекомендательную систему на основе данных о контенте. Будем работать с [датасетом](https://lms-cdn.skillfactory.ru/assets/courseware/v1/747dae7bf99b18ce3b24bd34aa7bc29b/asset-v1:SkillFactory+DSPR-2.0+14JULY2021+type@asset+block/netflix_titles.zip) (прямая ссылка), содержащим информацию об оценивании фильмов на платформе Netflix.

In [2]:
netflix_data = pd.read_csv('data/netflix_titles.zip')
print('Data shape:', netflix_data.shape)
netflix_data.head(3)

Data shape: (7787, 12)


,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,s1,TV Show,3%,NaN,"João Miguel, Bianca Comparato, Michel Gomes, R...",Brazil,"August 14, 2020",2020,TV-MA,4 Seasons,"International TV Shows, TV Dramas, TV Sci-Fi &...",In a future where the elite inhabit an island ...
1,s2,Movie,7:19,Jorge Michel Grau,"Demián Bichir, Héctor Bonilla, Oscar Serrano, ...",Mexico,"December 23, 2016",2016,TV-MA,93 min,"Dramas, International Movies",After a devastating earthquake hits Mexico Cit...
2,s3,Movie,23:59,Gilbert Chan,"Tedd Chan, Stella Chung, Henley Hii, Lawrence ...",Singapore,"December 20, 2018",2011,R,78 min,"Horror Movies, International Movies","When an army recruit is found dead, his fellow..."


Признаки в данных

|features|description|features|description|
|-|-|-|-|
|`show_id`|id фильма|`date_added`|дата добавления|
|`type`|его тип (фильм или сериал)|`release_year`|год выхода на экраны|
|`title`|название|`rating`|рейтинг|
|`director`|режиссер|`duration`|продолжительность|
|`cast`|актерский состав|`listened_in`|жанр(-ы)|
|`country`|страна|`description`|описание|

В первую очередь нам необходимо определить, на основании чего мы будем рассматривать близость фильмов. Выберем для этой задачи описание фильма, ведь в нём, скорее всего, содержится много информации. Однако описание — это текст. Есть много подходов к преобразованию текста в вектор, и мы будем использовать подход TF-IDF (Term Frequency-Inverse Document Frequency).

## Обработка данных

Учтём стоп-слова, т.е. предлоги и другие служебные части речи, которые не несут содержательной информации, и определим нашу модель:

In [3]:
model = TfidfVectorizer(stop_words='english')

Заполним пропуски пустыми строками:

In [4]:
netflix_data['description'] = netflix_data['description'].fillna('')

Трансформируем наши описания в матрицу:

In [5]:
feature_matrix = model.fit_transform(netflix_data['description'])

Посмотрим на размер получившейся матрицы:

In [6]:
print('Matrix shape:', feature_matrix.shape)

Matrix shape: (7787, 17905)


Теперь необходимо вычислить косинусную близость. Можно сделать это так:

In [7]:
cosine_sim = linear_kernel(feature_matrix)

Можно было, конечно воспользоватся [cosine_similarity](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.pairwise.cosine_similarity.html#sklearn.metrics.pairwise.cosine_similarity), но эта функция в нашем случае была бы избыточной, так как повторно реализовала бы нормировку векторов.

Вернём индексацию и уберём дубликаты из данных:

In [8]:
indices = pd.Series(netflix_data.index, index=netflix_data['title']).drop_duplicates()

## Реализация рекомендательной системы: начало

Теперь пропишем функцию для создания рекомендаций:

In [9]:
def get_recommendations(title):
    idx = indices[title]
    #вычисляем попарные коэффициенты косинусной близости
    scores = list(enumerate(cosine_sim[idx]))
    #сортируем фильмы на основании коэффициентов косинусной близости по убыванию
    scores = sorted(scores, key=lambda x: x[1], reverse=True)
    #выбираем десять наибольших значений косинусной близости; нулевую не берём, т. к. это тот же фильм
    scores =   scores[1:11]
    #забираем индексы
    ind_movie = [i[0] for i in scores]
    #возвращаем названия по индексам
    return netflix_data['title'].iloc[ind_movie]

Допустим, мы посмотрели фильм "Star Trek", и наша функция будет готова порекомендовать нам ещё несколько похожих фильмов:

In [10]:
get_recommendations('Star Trek')

5788             Star Trek: The Next Generation
5787                      Star Trek: Enterprise
5786                 Star Trek: Deep Space Nine
5557                     She's Out of My League
134                                  7 Days Out
6664                        The Midnight Gospel
6023                                     Teresa
4863    Pinkfong & Baby Shark's Space Adventure
5104                                       Rats
5970                             Tales by Light
Name: title, dtype: object

Или мы посмотрели детский фильм "Balto", вышедшего на экраны в 1995 году, тогда рекомендация будет выглядеть так:

In [11]:
get_recommendations('Balto')

709                Balto 2: Wolf Quest
7446                           Vroomiz
1338    Chilling Adventures of Sabrina
7388                          Vampires
1770                          Dinotrux
2767                     Hold the Dark
5540                 Shanghai Fortress
4041                             Mercy
2582                       Half & Half
1365        Christmas in the Heartland
Name: title, dtype: object

## Реализация рекомендательной системы: продолжение

Ну хорошо, мы смоделировали такую ситуацию, что потребитель контента просмотрел один фильм и получил предложение посмотреть ещё несколько других фильмов, описание которых похоже на описание просмотренного.

Но а что нам делать потом, когда потребитель просмотрит второй фильм? Делать рекомендацию лишь по последнему фильму было бы неразумно: вероятны повторные рекомендации одних и тех же фильмов. Есть выход: можно вычислить средний вектор для всех просмотренных фильмов, используя существующие векторы. Это позволит создать "профиль" пользователя на основе его предпочтений.

Но если количество просмотренных фильмов становится большим, использование простого среднего вектора может привести к тому, что старые фильмы будут влиять на профиль пользователя слишком сильно. Чтобы учесть временной фактор и дать больший вес недавно просмотренным фильмам, можно использовать подход с взвешиванием.

Вот один из способов реализовать это:

1. Присвоить каждому просмотренному фильму вес в зависимости от того, как давно он был просмотрен. Например, можно использовать экспоненциальное затухание, где более свежие фильмы имеют больший вес.

2. Вместо простого среднего вектора, вычислить взвешенное среднее.

In [12]:
def get_user_profile(viewed_data, feature_matrix):
    # Получаем индексы просмотренных фильмов
    titles = list(viewed_data.keys())
    indices = netflix_data[netflix_data['title'].isin(titles)].index
    
    # Извлекаем векторы для просмотренных фильмов
    viewed_vectors = feature_matrix[indices]
    
    # Текущая дата
    current_date = datetime.now()

    # Вычисляем разницу в днях
    viewed_dates = list(viewed_data.values())
    days_diff = np.array([(current_date - date).days for date in viewed_dates])

    # Присваиваем веса на основе даты просмотра (ближайшие - больший вес)
    weights = np.exp(-days_diff / 30)  # Делим на 30 для нормализации

    # Вычисляем взвешенное среднее
    weighted_average_vector = np.average(viewed_vectors.toarray(), axis=0, weights=weights)
    
    return weighted_average_vector, indices


def get_recommendations_from_profile(user_profile_vector, feature_matrix, viewed_indices):
    # Преобразуем user_profile_vector в нужный формат
    user_profile_vector_array = user_profile_vector.reshape(1, -1)

    # Вычисляем косинусное сходство между профилем пользователя и всеми фильмами
    scores = linear_kernel(user_profile_vector_array, feature_matrix.toarray())
    
    # Сортируем фильмы по убыванию сходства
    scores = list(enumerate(scores[0]))
    scores = sorted(scores, key=lambda x: x[1], reverse=True)
    
    # Возвращаем названия фильмов, исключая уже просмотренные
    recommended_indices = [i[0] for i in scores if i[0] not in viewed_indices]
    
    return netflix_data['title'].iloc[recommended_indices[:10]]

Посмотрим, что у нас получится:

In [18]:
# Данные о просмотрах
viewed_data = {
    'Balto': datetime(2024, 9, 1),
    'Star Trek': datetime(2024, 8, 1),
}

# Расчёт профильного вектора 
user_profile_vector, viewed_indices = get_user_profile(
    viewed_data, feature_matrix
    )
# Рекомендации
recommendations = get_recommendations_from_profile(
    user_profile_vector, feature_matrix, viewed_indices
    )
recommendations

709                                   Balto 2: Wolf Quest
1338                       Chilling Adventures of Sabrina
7446                                              Vroomiz
7388                                             Vampires
1770                                             Dinotrux
2767                                        Hold the Dark
433     Alpha and Omega: The Legend of the Saw Tooth Cave
6286                                      The Degenerates
5540                                    Shanghai Fortress
5788                       Star Trek: The Next Generation
Name: title, dtype: object

Как видим, разнесённые по времени на месяц просмотры будут по-разному влиять на свежие рекомендации.

Вот так простой векторизатор TF-IDF помогает реализовать content-based подход при разработке простенькой рекомендательной системы. Однако опора лишь на векторизацию текстового описания объектов для качественной генерации рекомендаций может быть недостаточно: если поменять в рассмотренном примере имена фильмов в словаре `viewed_data`, то результат не поменяется: это может происходить потому, что вектор признаков фильма "Star Trek" менее "уникален" в пространстве признаков по сравнению с "Balto"; но возможно, в нашей матрице признаков фильмы, похожие на "Balto", имеют более выраженные или уникальные характеристики, что и приводит к их доминированию в рекомендациях.

Вероятно, для устранения такого рода артефактов нужно использовать больше данных о фильмах.